In [1]:
import rospy
from moveit_python import *

rospy.init_node("moveit_py")
g = MoveGroupInterface("panda_arm", "panda_link0")
p = PlanningSceneInterface("panda_link0")

[INFO] [1565238142.681825]: Waiting for get_planning_scene


In [4]:
joints = []
for i in range(1,8):
    joints.append('panda_joint'+str(i))

res = g.moveToJointPosition(
                  joints, 
                  [ 1.00, 0.5, 
                    0.00, 0.00, 0.50, 
                    1.5, -1.5])


In [2]:
# reproduce pick and place tutorial with python

# environment setup
p.removeCollisionObject('table1')
p.removeCollisionObject('table2')
p.removeCollisionObject('object')
p.addBox("table1", 0.2,0.4,0.4, 0.5, 0.0, 0.2)
p.addBox("table2", 0.2,0.4,0.4, 0.0, 0.5, 0.2)
p.addBox("object", 0.02, 0.02, 0.2, 0.5, 0.0, 0.5)
print p.getKnownCollisionObjects()

[]


In [9]:
from moveit_msgs.msg import Grasp, PlaceLocation
from trajectory_msgs.msg import JointTrajectoryPoint
from tf.transformations import *

P = PickPlaceInterface("panda_arm", "hand")

grasp = Grasp()
grasp.grasp_pose.header.frame_id = "panda_link0"
q = quaternion_from_euler(-numpy.pi/2.0, -numpy.pi/4.0, -numpy.pi/2.0)
grasp.grasp_pose.pose.orientation.x = q[0]
grasp.grasp_pose.pose.orientation.y = q[1]
grasp.grasp_pose.pose.orientation.z = q[2]
grasp.grasp_pose.pose.orientation.w = q[3]
grasp.grasp_pose.pose.position.x = 0.415
grasp.grasp_pose.pose.position.y = 0.0
grasp.grasp_pose.pose.position.z = 0.5

grasp.pre_grasp_approach.direction.header.frame_id = "panda_link0"
grasp.pre_grasp_approach.direction.vector.x = 1.0
grasp.pre_grasp_approach.min_distance = 0.095
grasp.pre_grasp_approach.desired_distance = 0.115

grasp.post_grasp_retreat.direction.header.frame_id = "panda_link0";
grasp.post_grasp_retreat.direction.vector.z = 1.0
grasp.post_grasp_retreat.min_distance = 0.1
grasp.post_grasp_retreat.desired_distance = 0.25

#open gripper
grasp.pre_grasp_posture.joint_names = [
    "panda_finger_joint1",
    "panda_finger_joint2"
]

jtp = JointTrajectoryPoint()
jtp.positions = [0.04, 0.04]
jtp.time_from_start = rospy.Duration.from_sec(0.5)
grasp.pre_grasp_posture.points = [jtp]

# #close gripper
grasp.grasp_posture.joint_names = [
    "panda_finger_joint1",
    "panda_finger_joint2"
]
jtp2 = JointTrajectoryPoint()
jtp2.positions = [0.00, 0.00]
jtp2.time_from_start = rospy.Duration.from_sec(0.5)
grasp.grasp_posture.points = [jtp2]

# # print grasp
res = P.pickup("object", [grasp, ], support_name = "table1")


# L = PlaceLocation()
# # fill in l
# P.place("object", [L, ], goal_is_eef = True, support_name = "table2")

In [10]:
location = PlaceLocation()

location.place_pose.header.frame_id = "panda_link0"
q = quaternion_from_euler(0.0, 0.0, numpy.pi/2.0)
location.place_pose.pose.orientation.x = q[0]
location.place_pose.pose.orientation.y = q[1]
location.place_pose.pose.orientation.z = q[2]
location.place_pose.pose.orientation.w = q[3]
location.place_pose.pose.position.x = 0.0
location.place_pose.pose.position.y = 0.5
location.place_pose.pose.position.z = 0.5

location.pre_place_approach.direction.header.frame_id = "panda_link0"
location.pre_place_approach.direction.vector.z = -1.0
location.pre_place_approach.min_distance = 0.095
location.pre_place_approach.desired_distance = 0.115

location.post_place_retreat.direction.header.frame_id = "panda_link0";
location.post_place_retreat.direction.vector.y = -1.0
location.post_place_retreat.min_distance = 0.1
location.post_place_retreat.desired_distance = 0.25

#open gripper
location.post_place_posture.joint_names = [
    "panda_finger_joint1",
    "panda_finger_joint2"
]

jtp = JointTrajectoryPoint()
jtp.positions = [0.04, 0.04]
jtp.time_from_start = rospy.Duration.from_sec(0.5)
location.post_place_posture.points = [jtp]

res = P.place("object", [location, ], support_name = "table2")